In [9]:
import torch

lr = torch.rand((32, 32))
hr = torch.rand((32, 1, 128, 128))
elev = torch.rand((32, 1, 128, 128))

concatenated = torch.cat((hr, elev), dim=1)

In [10]:
concatenated.shape

torch.Size([32, 2, 128, 128])